# Uncertainty estimation in deep learning based-classifiers of High Energy Physics events using Monte Carlo Dropout.
-----
## Higgs Dataset

R. Pezoa (UV, CCTVal-USM), S. Bórquez(USM), W. Brooks (USM), L. Salinas (USM), C. Torres (USM)

## Libraries

In [ ]:
import time

import tqdm
import tensorflow as tf
import tensorflow_addons as tfa
import autokeras as ak
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import utils
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten,  Input
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from mc_dropout import *

tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7120)])
    except RuntimeError as e:
        print(e)

## Data
-----
Hggs dataset

- Data is obtained from: https://www.openml.org/d/23512
- Each event is represented by a set of 28 features, including 21 low-level features corresponding to physics properties measured by the detector, and 7 high-level features derived from the previous ones.

In [ ]:
data_path = "/mnt/storage-large/dataset/higgs/phpZLgL9q.csv"
#data_path = "/mnt/storage-large/dataset/higgs/HIGGS.csv"

In [ ]:
seed_=420
# Read data file
df = pd.read_csv(data_path)
df.rename(columns = {'class': 'label'}, inplace = True)
# Removing last row containinng "?" values
df.drop(df.tail(1).index,inplace=True) # drop last n rows
df = df.apply(pd.to_numeric)
# Pandas dataframe for correlation matrix without label column
df_corr = df.drop('label', inplace=False, axis=1)

# Scaling data
y = df["label"]
X = df.iloc[:,1:]

scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)
df_scaled = pd.DataFrame(scaled_data, columns=X.columns)

# Features names
features_names = list(X.columns)

# Training, validation, and testing data
X_train, X_test, y_train, y_test = train_test_split(scaled_data, y, test_size=0.2, random_state=seed_)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, shuffle = True, test_size=0.2, random_state=seed_)

In [ ]:
print("# X_train: %s" % (X_train.shape[0]))
print("# X_val: %s" % (X_val.shape[0]))
print("# X_test: %s" % (X_test.shape[0]))

In [ ]:
y_train = y_train.values.astype(float)
y_test = y_test.values.astype(float)

In [ ]:
X_val.shape

In [ ]:
X_train.shape

In [ ]:
y_train.shape

## Feature Engineering
---


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer

In [ ]:
#rf = RandomForestClassifier(max_depth=15)
#start_time = time.time()
#rf.fit(X_train, y_train)
#elapsed_time = time.time() - start_time
#print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")
#accuracy_score(y_val, rf.predict(X_val))

In [ ]:
#func = FunctionTransformer(np.expm1)
#X_exp = np.hstack((X,func.fit_transform(X)))

#poly = PolynomialFeatures(2)
#X_poly = poly.fit_transform(X_exp)
#X_poly = poly.fit_transform(X)

scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)

# Training, validation, and testing data
X_train, X_test, y_train, y_test = train_test_split(scaled_data, y, test_size=0.2, random_state=seed_)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, shuffle = True, test_size=0.2, random_state=seed_)

In [ ]:
X.shape

## Hyperparameters
--- 
We will use AutoKeras

https://autokeras.com/tutorial/structured_data_classification/

https://autokeras.com/structured_data_classifier/

In [ ]:
batch_size = 8
num_classes = 2
epochs = 1 #20
max_trials =  1 #2000 # tries n different models.
metrics = ['accuracy',  tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]

In [ ]:
from keras_tuner.engine import hyperparameters


class DensePReLUBlock(ak.DenseBlock):
    def build(self, hp, inputs=None):
        inputs = tf.nest.flatten(inputs)
        ak.utils.utils.validate_num_inputs(inputs, 1)
        input_node = inputs[0]
        output_node = input_node
        output_node = ak.blocks.reduction.Flatten().build(hp, output_node)

        use_batchnorm = self.use_batchnorm
        if use_batchnorm is None:
            use_batchnorm = hp.Boolean("use_batchnorm", default=False)

        for i in range(ak.utils.utils.add_to_hp(self.num_layers, hp)):
            units = ak.utils.utils.add_to_hp(self.num_units, hp, "units_{i}".format(i=i))
            output_node = tf.keras.layers.Dense(units)(output_node)
            if use_batchnorm:
                output_node = tf.keras.layers.BatchNormalization()(output_node)
            output_node = tf.keras.layers.PReLU(output_node)  # I changed this activation function only
            if ak.utils.utils.add_to_hp(self.dropout, hp) > 0:
                output_node = tf.keras.layers.Dropout(ak.utils.utils.add_to_hp(self.dropout, hp))(
                    output_node
                )
        return output_node


def get_automodel(max_trials, num_classes, metrics, column_names, overwrite, objective):    
    input_node = ak.StructuredDataInput(column_names=column_names)
    #output_node = DensePReLUBlock(
    output_node = ak.DenseBlock(
        num_layers=hyperparameters.Choice("num_layers", [3, 4,  5, 6], default=3),
        num_units=hyperparameters.Choice("num_units", [32, 64, 128, 256, 512], default=128)
    )(input_node)
    output_node = ak.ClassificationHead(num_classes=num_classes, metrics=metrics)(output_node)
    auto_model = ak.AutoModel(
        inputs=input_node, outputs=output_node, overwrite=overwrite, max_trials=max_trials, objective=objective
    )
    return auto_model

In [ ]:
# Initialize the structured data classifier.
auto_model = get_automodel(
    max_trials=max_trials,
    column_names=features_names,
    num_classes=num_classes,
    metrics=metrics,
    objective='val_accuracy',
    overwrite=True,
)

In [ ]:
# Feed the structured data classifier with training data.
h = auto_model.fit(
    x=X_train, y=y_train, validation_data=(X_val, y_val), epochs=epochs, verbose=2#, #callbacks=callbacks
)

In [ ]:
auto_model.evaluate(X_val, y_val);

In [ ]:
auto_model.evaluate(X_test, y_test);

In [ ]:
model = auto_model.export_model()
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.save('model')

In [ ]:
mc_model = get_mc_model(model, metrics)
mc_model.summary()

In [ ]:
tf.keras.utils.plot_model(mc_model, show_shapes=True)

In [ ]:
mc_model.save('mc_model')

## Traditional model

In [ ]:
model = tf.keras.models.load_model('model')

In [ ]:
h = model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=30,
              verbose=1,
              validation_data=(X_test, y_test))

In [ ]:
plt.figure()
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.show()

In [ ]:
plt.figure()
plt.plot(h.history['accuracy'])
#plt.plot(h.history['val_accuracy'])
plt.show()

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Bayesian Deep Learning

In [ ]:
mc_model = tf.keras.models.load_model('mc_model')

In [ ]:
h_mc = mc_model.fit(X_train, y_train,
                    batch_size=16,
                    epochs=5,
                    verbose=1,
                    validation_data=(X_test, y_test))

In [ ]:
plt.figure()
plt.plot(h_mc.history['loss'], label="train loss")
plt.plot(h_mc.history['val_loss'], label="val loss")
plt.xlabel("iteration")
plt.ylabel("loss")
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(h_mc.history['accuracy'], label = "train acc")
plt.plot(h_mc.history['val_accuracy'], label = "val acc")
plt.legend()
plt.show()

## Forward Pass

In [ ]:
T = 500

In [ ]:
mc_predictions = np.array(
    [
    mc_model.predict_on_batch(X_test)
    for i in tqdm.tqdm(range(T))
    ]

)

In [ ]:
mc_predictions.shape  # (T, batch, 1)

In [ ]:
accs = []
precs = []
recs = []
f1s = []
for y_p in mc_predictions:
    # Select predicted class
    y_p_class = proba_to_class(y_p)
    acc = accuracy_score(y_test, y_p_class)
    prec = precision_score(y_test, y_p_class)
    rec = recall_score(y_test, y_p_class)
    f1 = f1_score(y_test, y_p_class)
    accs.append(acc)
    precs.append(prec)
    recs.append(rec)
    f1s.append(f1)
print("MC accuracy: {:.1%}".format(sum(accs)/len(accs)))
print("MC precision: {:.1%}".format(sum(precs)/len(precs)))
print("MC recall: {:.1%}".format(sum(recs)/len(recs)))
print("MC f1: {:.1%}".format(sum(f1s)/len(f1s)))

In [ ]:
mc_ensemble_pred = predictive_distribution(mc_predictions)

mc_ensemble_pred_class = proba_to_class(mc_ensemble_pred)


ensemble_acc = accuracy_score(y_test, mc_ensemble_pred_class)
ensemble_f1 = f1_score(y_test, mc_ensemble_pred_class)
print("MC-ensemble accuracy: {:.1%}".format(ensemble_acc))
print("MC-ensemble f1-score: {:.1%}".format(ensemble_f1))

In [ ]:
plt.style.use("ggplot")
plt.hist(accs);
plt.axvline(x=ensemble_acc, color="b");
plt.title("accuracy distribution on testing data") 

In [ ]:
plt.style.use("ggplot")
plt.hist(f1s);
plt.title("F1 distribution on testing data") 
plt.axvline(x=ensemble_f1, color="b");

## Some tests on specific data

In [ ]:
idx=0 # taking data with index idx
# mc_predictions, a list with 500 elements, because we have 500 forward passes:
# each element is a numpy array of size (n_tests, 2)

# this line is taking the test data with idx=1 of each forward pass, and putting it into p0
p0 = mc_predictions[:, idx]
p0_predictive_distribution = predictive_distribution(p0)

# these are examples of highly uncertain prediction
#p0 = np.random.uniform(0, 1, (500,1))
#p0_predictive_distribution = predictive_distribution(p0)

#p0 = np.where(np.random.uniform(0, 1, (500,1)) > 0.5, 1, 0)
#p0_predictive_distribution = predictive_distribution(p0)

# this is a example of low uncertain prediction
#p0 = np.zeros((500,1))
#p0_predictive_distribution = predictive_distribution(p0)

# mean of the prediction in data with idx=1
print("predictive distribution: {:.2%}".format(p0_predictive_distribution))
print("posterior mean: {}".format(p0_predictive_distribution > 0.5))
print("true label: {}".format(int(np.array(y_test)[idx])))

## Computing variance

In [ ]:
print("class: {}; proba: {:.2%}; var: {:.2%} ".format(0, (1 - p0).mean(), (1-p0).std()))
print("class: {}; proba: {:.2%}; var: {:.2%} ".format(1, p0.mean(), p0.std()))

In [ ]:
plt.style.use("ggplot")
plt.hist(p0, bins=20);
plt.title("Samples distribution on a test instance") 
plt.axvline(x=p0_predictive_distribution, color="b", label='Predictive distribution');
plt.legend()
plt.xlabel('$p(y|x,w_t)$')
plt.xlim([0,1]);


## Computing mutual information
-----

In [ ]:
mutual_information(mc_ensemble_pred, mc_predictions)

In [ ]:
mutual_information(p0_predictive_distribution, p0, is_sample=True, normalize=True)

## Computing predictive entropy
-----

In [ ]:
predictive_entropy(mc_ensemble_pred)

In [ ]:
predictive_entropy(p0_predictive_distribution, is_sample=True, normalize=True)